# FedLib: Heterogeneous Federated Learning using Dynamic Model Pruning and Adaptive Gradient

## Importing supportive libaries
This notebook shows a demo on PyTorch back-end model impelementation.

In the very begining, we import the supporting libraries.

In [1]:
import torch
import numpy as np
import copy

from fedlib.ve import FEDDFEnv as Simulator
from fedlib.lib import Server, Client
from fedlib.networks import resnet20
from fedlib.lib.sampler import random_sampler
from fedlib.lib.algo import feddp
from fedlib.datasets import partition_data, get_dataloader,get_client_dataloader
from fedlib.utils import get_logger

## Define arguments
Here we define arguments. To show an intuitive example, we show the demo store all the parameters in a dictionary in the following code block.
We also provide APIs for you create your arguments in a `*.yaml` file.

In [2]:
args = {}
args["n_clients"] = 2
args["device"] = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args['sample_fn'] = random_sampler
args['trainer'] = feddp(logger=get_logger())
args['communicator'] = None
args["test_dataset"] = None
args["partition"] = "noniid-labeldir"
args["dataset"] = "cifar10"
args["datadir"] = "./data"
args["beta"] = 0.5
args["batch_size"] = 64
args["global_model"] = resnet20()
args["lr"] = 0.01
args["optimizer"] = "SGD"
args["lr_scheduler"] = "ExponentialLR"

Load test dataset for server, and passing it as an argument

In [3]:
X_train, y_train, X_test, y_test, net_dataidx_map, traindata_cls_counts = partition_data(
    args["dataset"], args["datadir"], args['partition'], args['n_clients'], beta=args['beta'])
n_classes = len(np.unique(y_train))
train_dl_global, test_dl_global, train_ds_global, test_ds_global = get_dataloader(args["dataset"],
                                                                                    args["datadir"],
                                                                                      args["batch_size"],
                                                                                      32)
args["test_dataset"] = test_dl_global

Files already downloaded and verified
Files already downloaded and verified


INFO:root:Data statistics: {0: {0: 1016, 1: 1431, 2: 4538, 3: 3328, 4: 1677, 5: 2289, 6: 3536, 7: 1113, 8: 4970, 9: 4629}, 1: {0: 3984, 1: 3569, 2: 462, 3: 1672, 4: 3323, 5: 2711, 6: 1464, 7: 3887, 8: 30, 9: 371}}


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Create server and clients objects
Here we use the arguments we defined before, and create server and clients.

In [4]:
server = Server(**args)
clients = {}

data_loaders, test_loaders = get_client_dataloader(args["dataset"], args["datadir"], args['batch_size'], 32, net_dataidx_map)


Files already downloaded and verified
Files already downloaded and verified
key: 0 	, dataid 28527 train_ds: 22821 test_ds: 5706
Files already downloaded and verified
Files already downloaded and verified
key: 1 	, dataid 21473 train_ds: 17178 test_ds: 4295
Total train: 39999 	 Total test: 10001


In [5]:


for id in range(args["n_clients"]):
    # dataidxs = net_dataidx_map[id]
    args["id"] = id
    # args["trainloader"], _, _, _ = get_dataloader(args["dataset"], args["datadir"], args['batch_size'], 32, dataidxs)
    args["trainloader"] = data_loaders[id]
    args["testloader"] =test_loaders[id]
    args["model"] = copy.deepcopy(args["global_model"])
    args['criterion'] = torch.nn.CrossEntropyLoss()
    clients[id] = Client(**args)

    


Client: 0 	Train: 22821 	Test: 5706
Client: 1 	Train: 17178 	Test: 4295


## Create simulator

Simulator simulates the virtual federated learning environments, and run server and clients on single device.

In [6]:
simulator = Simulator(server=server, clients=clients, communication_rounds=10,n_clients= 2,sample_rate=1)

## Run simulator
User API Simulator.run

In [7]:
simulator.run(local_epochs=2,pruning_threshold=1e-3)

INFO:root:*******starting rounds 1 optimization******
INFO:root:optimize the 1-th clients


[1 0]


/Users/sixingyu/opt/anaconda3/envs/fedlib/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
INFO:root:Layer: conv1, Sparsity: 0.23%
INFO:root:Layer: layer1.0.conv1, Sparsity: 0.56%
INFO:root:Layer: layer1.0.conv2, Sparsity: 0.65%
INFO:root:Layer: layer1.1.conv1, Sparsity: 0.52%
INFO:root:Layer: layer1.1.conv2, Sparsity: 0.65%
INFO:root:Layer: layer1.2.conv1, Sparsity: 0.56%
INFO:root:Layer: layer1.2.conv2, Sparsity: 0.61%
INFO:root:Layer: layer2.0.conv1, Sparsity: 0.52%
INFO:root:Layer: layer2.0.conv2, Spars

Layer: conv1, Sparsity: 0.23%
Layer: layer1.0.conv1, Sparsity: 0.56%
Layer: layer1.0.conv2, Sparsity: 0.65%
Layer: layer1.1.conv1, Sparsity: 0.52%
Layer: layer1.1.conv2, Sparsity: 0.65%
Layer: layer1.2.conv1, Sparsity: 0.56%
Layer: layer1.2.conv2, Sparsity: 0.61%
Layer: layer2.0.conv1, Sparsity: 0.52%
Layer: layer2.0.conv2, Sparsity: 0.82%
Layer: layer2.1.conv1, Sparsity: 1.04%
Layer: layer2.1.conv2, Sparsity: 1.01%
Layer: layer2.2.conv1, Sparsity: 1.09%
Layer: layer2.2.conv2, Sparsity: 0.89%
Layer: layer3.0.conv1, Sparsity: 0.88%
Layer: layer3.0.conv2, Sparsity: 1.39%
Layer: layer3.1.conv1, Sparsity: 1.36%
Layer: layer3.1.conv2, Sparsity: 1.39%
Layer: layer3.2.conv1, Sparsity: 1.29%
Layer: layer3.2.conv2, Sparsity: 1.38%
Model sparsity: 0.89%


IndexError: Replacement index 2 out of range for positional args tuple

array([1])